### Problem 1 - The Multi-Head Attention Score


In the Transformer paper ("Attention is All You Need"), the core math is:
$$\text{Score} = QK^T$$
Where $Q$ and $K$ are 4D tensors: `(Batch, Heads, Sequence_Length, Features)`. 

###  Setup


```python
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import os



# Create Toy Data (Research Scale)
# B=2 (Batch size), H=4 (Heads), L=10 (Seq Length), D=32 (Feature Dim)
B, H, L, D = 2, 4, 10, 32
Q = torch.randn(B, H, L, D)
K = torch.randn(B, H, L, D)
```

---

###  Challenge: Implementation

You need to calculate the attention score matrix of shape `(B, H, L, L)`. 

1.  **Method A (The "Messy" Way):** Use `torch.matmul`. 
2.  **Method B (The "Better" Way):** Use `torch.einsum`.
   


<details>
<summary>Hint</summary>

1.  **Method A**
    *   *Hint:* You cannot multiply `(B, H, L, D)` by `(B, H, L, D)`. You must transpose the second tensor to `(B, H, D, L)` first.
2.  **Method B** 
    *   *Hint:* The formula is `'bhld, bhmd -> bhlm'`. 
    *   **Logic:** `b` and `h` stay the same. `l` is the row of Q, `m` is the row of K. `d` is the dimension that gets "multiplied and summed" (the dot product).

</details>

In [11]:
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter


writer = SummaryWriter('runs/attention_experiment')

B, H, L, D = 2, 4, 10, 32
Q = torch.randn(B, H, L, D)
K = torch.randn(B, H, L, D)

#using matmul
attention_scores_matmul = torch.matmul(
        Q, K.transpose(-2, -1)
)

print(f'Attention scores using matmul: {attention_scores_matmul.size()}')

#using einsum
attention_scores_einsum = torch.einsum(
    "bhld,bhmd->bhlm", Q, K
)


print(f'Attention scores using einsum: {attention_scores_einsum.size()}')

# Verification
matches = torch.allclose(attention_scores_matmul, attention_scores_einsum, atol=1e-6)
print(f"Do the methods match? {matches}")

Attention scores using matmul: torch.Size([2, 4, 10, 10])
Attention scores using einsum: torch.Size([2, 4, 10, 10])
Do the methods match? True


<details>
<summary>Post-Mortem (Problem 1)</summary>



####  Matmul vs. Einsum
*   **The Matmul Way:** `torch.matmul(Q, K.transpose(-2, -1))`
    *   **Pros:** Familiar to those coming from linear algebra.
    *   **Cons:** You have to remember *which* dimensions to transpose. If you have a 5D tensor (e.g., `[B, H, T, L, D]`), `transpose(-2, -1)` might not be what you want. It is **positional**.
*   **The Einsum Way:** `torch.einsum("bhld,bhmd->bhlm", Q, K)`
    *   **Pros:** It is **declarative**. You are stating: "Take dimension $d$ from the first and dimension $d$ from the second, multiply them, and sum them up." You don't care where $d$ is located; you just name it.
    *   **Cons:** Small learning curve for the notation string.


####  Connection to Strides
When you called `K.transpose(-2, -1)`, did PyTorch move data? **No.** (Refer back to Notebook 01). It just swapped the strides. 
When `torch.matmul` runs, it sees that `K` is non-contiguous and handles the internal memory jumps. 
 `einsum` often optimizes the operation internally (using the `opt_einsum` logic), sometimes making it faster than multiple `transpose` and `matmul` calls for complex operations.

</details>

___

### Problem 2: The Pairwise Distance Kernel (Broadcasting)

 let’s tackle the **Expansion Trick**. This is used in papers involving **Contrastive Learning** (like CLIP or SimCLR), where you compare every image feature in a batch to every text feature.

**The Setup:**
```python
# A: 5 points in 2D space (e.g., Target centroids)
A = torch.tensor([
    [0.0, 0.0],
    [1.0, 1.0],
    [2.0, 2.0],
    [3.0, 3.0],
    [4.0, 4.0]
])

# B: 3 points in 2D space (e.g., New data points)
B = torch.tensor([
    [0.5, 0.5],
    [2.5, 2.5],
    [10.0, 10.0]
])
```

**Your Challenge:**
Compute a `(5, 3)` matrix where `output[i, j]` is the Squared Euclidean Distance between `A[i]` and `B[j]`.

1.  **Requirement:** Do NOT use `for` loops.
2.  **Requirement:** Use broadcasting with the `None` (unsqueeze) trick.
3.  **The Formula:** $(x_1 - x_2)^2 + (y_1 - y_2)^2$

In [12]:
# A: 5 points in 2D space (e.g., Target centroids)
A = torch.tensor([
    [0.0, 0.0],
    [1.0, 1.0],
    [2.0, 2.0],
    [3.0, 3.0],
    [4.0, 4.0]
])

# B: 3 points in 2D space (e.g., New data points)
B = torch.tensor([
    [0.5, 0.5],
    [2.5, 2.5],
    [10.0, 10.0]
])

print(f"A.shape: {A.shape}, B.shape: {B.shape}")

A.shape: torch.Size([5, 2]), B.shape: torch.Size([3, 2])


In [14]:
#  Expand dimensions using 'None'
# A[:, None, :] -> (5, 1, 2)
# B[None, :, :] -> (1, 3, 2)
diff = A[:, None, :] - B[None, :, :]

#  Square the differences
# Shape stays (5, 3, 2)
sq_diff = diff ** 2

# Sum over the feature dimension (the last dimension, dim=2)
# (5, 3, 2) -> (5, 3)
dist_matrix = sq_diff.sum(dim=2)

print(f"Distance Matrix Shape: {dist_matrix.shape}")
print(dist_matrix)

Distance Matrix Shape: torch.Size([5, 3])
tensor([[  0.5000,  12.5000, 200.0000],
        [  0.5000,   4.5000, 162.0000],
        [  4.5000,   0.5000, 128.0000],
        [ 12.5000,   0.5000,  98.0000],
        [ 24.5000,   4.5000,  72.0000]])


<details>
<summary>Post-Mortem (Problem 2)</summary>

**Q1: What is the shape of the intermediate tensor `diff`?**
*   **Answer:** $(5, 3, 2)$.
*   **Deep Dive:** In  memory (RAM), PyTorch just created $5 \times 3 \times 2 = 30$ floating-point numbers. 

**Q2: The Memory Warning**
Imagine you are implementing a research paper with a batch size of 1000 images, and each image has 512 features.
*   $A$ is $(1000, 512)$
*   $B$ is $(1000, 512)$
*   `A[:, None, :] - B[None, :, :]` creates a tensor of $(1000, 1000, 512)$.
*   That is **512 million numbers**. At 4 bytes per float, that is **2 GB of GPU RAM** just for one subtraction!
*   **Lesson:** Broadcasting is beautiful, but if your dimensions are large, you will hit an `Out of Memory (OOM)` error. This is why researchers often use the formula $(a-b)^2 = a^2 + b^2 - 2ab$ with `torch.matmul`, because it avoids that huge 3D intermediate tensor.

</details>

___

### Problem 3: Manual Convolution (Unfold / Sliding Windows)

 In many research papers (like **Swin Transformers** or **Graph Neural Networks**), you don't use standard Convolutions. You need to extract "patches" or "neighborhoods."

 A convolution is just a dot product over a sliding window. To do this efficiently, we "unroll" the windows into a matrix.

**The Setup:**
We have a 1D signal (representing time or a flattened image).
```python
# A signal of 10 values
signal = torch.tensor([10., 20., 30., 40., 50., 60., 70., 80., 90., 100.])
```

**Your Challenge:**
Use the `.unfold()` method to create a sliding window view.
1.  **Window Size:** 3
2.  **Stride:** 1 (The window moves 1 step at a time)
3.  **Goal:** Transform `signal` into a 2D tensor where each row is a window:
    ```text
    [[10, 20, 30],
     [20, 30, 40],
     [30, 40, 50],
     ...]
    ```
4.  **The "Math" Challenge:** Once you have the windows, define a kernel `torch.tensor([0.5, 1.0, 0.5])` and compute the convolution (dot product of each window with the kernel) using **Einsum**.


In [ ]:
import torch

# Our Signal
signal = torch.tensor([10., 20., 30., 40., 50., 60., 70., 80., 90., 100.])

# The Unfold operation
# .unfold(dimension, size, step)
# dimension=0: we are unfolding the only dimension we have
# size=3: each window has 3 elements
# step=1: move the window 1 element at a time
windows = signal.unfold(0, 3, 1)

print(f"Original Signal Shape: {signal.shape}")
print(f"Unfolded Windows Shape: {windows.shape}")
print("First 3 windows:\n", windows[:3])

# The Kernel (for convolution)
kernel = torch.tensor([0.5, 1.0, 0.5])


# Compute the convolution using einsum. 
# 'windows' of shape (8, 3) and 'kernel' of shape (3).
# We want to multiply each window by the kernel and sum them up.
# 'ij, j -> i' 
# (i is the number of windows, j is the window size)

conv_result = torch.einsum('ij, j -> i', windows, kernel)
print(f"Convolution Result Shape: {conv_result.shape}")
print("Convolution Result:\n", conv_result)

Original Signal Shape: torch.Size([10])
Unfolded Windows Shape: torch.Size([8, 3])
First 3 windows:
 tensor([[10., 20., 30.],
        [20., 30., 40.],
        [30., 40., 50.]])
Convolution Result Shape: torch.Size([8])
Convolution Result:
 tensor([ 40.,  60.,  80., 100., 120., 140., 160., 180.])


<details>
<summary>Post-Mortem (Problem 3)</summary>


In Problem 1, we saw that `transpose` changes strides. `unfold` is even more extreme.
*   Original `signal` stride: `(1,)`
*   `unfolded` windows stride: `(1, 1)`

 Usually, to go to the next "row" in a matrix, you have to skip many elements. But here, to go to the next "window," you only skip **1** element. This is why the windows overlap! If you changed the `step` to **3**, the stride would become `(3, 1)`, meaning "skip 3 to get to the next window." This would create **non-overlapping** patches (used in Vision Transformers).


Deep Learning libraries don't actually slide a window in a `for` loop. They use a function called `im2col` (image to column), which is essentially a 2D version of `unfold`. It turns the image into a giant matrix of patches so that the convolution can be calculated as one single, massive **Matrix Multiplication** (which GPUs are incredibly fast at).
.

</details>

___